In [1]:

import os
import json
import requests
import numpy as np 
import pandas as pd
from sklearn.svm import SVR
from datetime import timedelta, datetime
import plotly.graph_objs as go
from dotenv import load_dotenv
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.datasets import make_regression, make_swiss_roll
from sklearn.ensemble import ExtraTreesRegressor, AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [2]:
#API call
def loadData():
    # load_dotenv('apiKeys.env')
    # api_key = os.getenv("extraKey") 
    # functionType = 'TIME_SERIES_DAILY'
    # interval = 'Daily'
    # outputsize = 'full'
    # ticker = input("Enter a stock ticker: ").upper()
    # url = f"https://www.alphavantage.co/query?function={functionType}&symbol={ticker}&outputsize={outputsize}&apikey={api_key}"
    # print(url)
    # # Get the response
    # response = requests.get(url)
    # # Retrieve reviews
    # reviews = response.json()
    # # Convert to a data frame
    # reviews_df = pd.json_normalize(reviews[f"Time Series ({interval})"])
    # # Extracting the time series data
    # time_series_data = reviews_df[f"Time Series ({interval})"]
    # # Convert to DataFrame
    # df = pd.DataFrame(time_series_data).T
    # # Rename columns
    # df.columns = ["open", "high", "low", "close", "volume"]
    # # Convert data types
    # df = df.astype({"open": float, "high": float, "low": float, "close": float, "volume": int})
    # # Convert index to datetime
    # df.index = pd.to_datetime(df.index)
    # # Sort index
    # df.sort_index(inplace=True)
    # df.index.name = 'date'
    # df.reset_index()
    # df['tomorrow'] = df['close'].shift(-1)
    # df['target'] = (df['tomorrow'] > df['close']).astype(int)
    # df.reset_index(inplace=True)
    # df['date'] = pd.to_datetime(df['date'])
    ticker = "IBM"
    df = pd.read_csv('ibm.csv')
    return df, ticker

In [3]:
# Prediction Function
def regression_predictions(df):
    X=df.drop(columns=['target','tomorrow','close','date'], axis=1)
    y1=df['close']
    X1_train, X1_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.3, random_state=42)
    
    #fit training data
    model = LinearRegression()
    model.fit(X1_train,y1_train)
    
    # Make predictions on testing data
    y_pred = model.predict(X)
    
    df['y_pred_LR'] = y_pred
    
    model = ExtraTreesRegressor()
    model.fit(X1_train,y1_train)
    
    # Make predictions on testing data
    y_pred = model.predict(X)
    
    df['y_pred_ETR'] = y_pred
    
    model = RandomForestRegressor()
    model.fit(X1_train,y1_train)
    
    # Make predictions on testing data
    y_pred = model.predict(X)
    
    df['y_pred_RFG'] = y_pred
    
    model = AdaBoostRegressor()
    model.fit(X1_train,y1_train)
    
    # Make predictions on testing data
    y_pred = model.predict(X)
    
    df['y_pred_ABR'] = y_pred
    
    return df

In [4]:
def graph_preds(df, ticker):
    # Visualization
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['date'], y=df['y_pred'], mode='lines', name='Predicted Price'))
    fig.add_trace(go.Scatter(x=df['date'], y=df['close'], mode='lines', name='Actual Price'))
    # fig.add_trace(go.Scatter(x=df['date'], y=df['y_pred_LR'], mode='lines', name='LR model'))
    fig.update_layout(title=f'Stock Prices and Predictions for {ticker}',
                      xaxis_title='Date',
                      yaxis_title='Price')
    fig.show()

In [8]:
def preds_output(df):
    num_shares = int(input("Enter the number of shares owned: "))
    investment_amount = float(input("Enter how much you have to invest: "))
    trend = 'higher' if df['close'].iloc[-1] > df['y_pred'].iloc[-1] else 'lower'
    action = 'buy' if trend == 'higher' else 'sell'
    optimal_volume = min(investment_amount / df['y_pred'].iloc[-1], num_shares)
    investment_value = df['close'].iloc[-1] * num_shares  # This is speculative
    
    print(f"Future predicted price for tomorrow: {df['close'].iloc[-1]}")
    print(f"The trend for the future price is {trend}.")
    print(f"You should {action}.")
    print(f"The optimal volume to {action} is {optimal_volume} shares.")
    print(f"The speculative future value of your investment is: ${investment_value:.2f}.")
    print("Please note, this is a prediction based on historical data and carries risk.")

In [6]:
alpha_df, ticker = loadData()

In [9]:
prediction_df = regression_predictions(alpha_df)
prediction_df['date'] = pd.to_datetime(prediction_df['date'])
prediction_df = prediction_df[prediction_df['date'].dt.year >= 2019]

prediction_df['y_pred'] = prediction_df[['y_pred_LR', 'y_pred_ETR', 'y_pred_RFG']].mean(axis=1)
# display(prediction_df.tail())
graph_preds(prediction_df, ticker)

preds_output(prediction_df)

/var/folders/q0/kyk7n6qs3cj9s3l8y_40h1rr0000gn/T/ipykernel_50770/1312011402.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/currygardner/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Future predicted price for tomorrow: 193.06
The trend for the future price is higher.
You should buy.
The optimal volume to buy is 2.5899247496719187 shares.
The speculative future value of your investment is: $19306.00.
Please note, this is a prediction based on historical data and carries risk.
